<a href="https://colab.research.google.com/github/anshupandey/Working_with_Large_Language_models/blob/main/WWL_C6_transformer_based_classification_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning: Transformer based Vectorizer

In [1]:
!pip install sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 9.6 MB/s eta 0:00:00


In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# load dataset
url = "https://raw.githubusercontent.com/anshupandey/Working_with_Large_Language_models/main/sentimentdata.csv"
df = pd.read_csv(url)
df.shape

(20, 2)

In [4]:
df.head()

,document,label
0,Pizza is great and I love pizza.,Positive
1,I hate burger and its bad to eat burger.,Negative
2,I hate dirty tables.,Negative
3,Burger is amazing and I love it more than anyt...,Positive
4,My boss is a monster and I hate him,Negative


In [5]:
x = df['document']
y = df['label']


## Exploring sentence vectorizer

In [6]:
# Load the pre-trained SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
vec = model.encode("Hello world, how are you, i am fine, how are you doing today.")
vec

array([ 1.28388088e-02,  1.86363328e-02,  6.62676841e-02, -1.37063600e-02,
       -1.83961838e-02, -8.53618160e-02,  8.16623420e-02, -7.99789734e-04,
       -9.43339467e-02, -1.06532057e-03, -1.36417169e-02,  2.55969092e-02,
       -1.72841009e-02,  3.20812338e-03, -2.35246681e-02, -5.00255600e-02,
        4.78703668e-03, -6.04917444e-02, -1.80326238e-01,  8.81443620e-02,
        1.31528843e-02,  6.34835586e-02,  3.56035605e-02,  7.79398158e-02,
       -1.13838978e-01,  5.72112994e-03, -3.35579971e-03, -2.73815468e-02,
        1.36863654e-02, -7.61968940e-02, -8.82716626e-02,  3.34809683e-02,
        2.36121025e-02, -4.95617129e-02, -3.69700268e-02,  1.42161418e-02,
       -1.37249315e-02, -1.32387340e-01, -1.71359833e-02,  2.41001253e-03,
        5.71570136e-02, -6.41963780e-02, -2.63626277e-02, -3.99268605e-03,
        4.52399515e-02, -2.96444912e-02, -1.81181612e-03,  4.05729488e-02,
        1.12659290e-01, -2.68510748e-02, -9.46810618e-02,  5.17554060e-02,
       -3.84240784e-02,  

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Example vectors
vec1 = model.encode("Noodles are good")
vec2 = model.encode("Noodles are amazing")
print(vec1.shape,vec2.shape)


# Calculate cosine similarity
similarity = cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))
print(f"Cosine Similarity: {similarity[0][0]}")


(384,) (384,)
Cosine Similarity: 0.9152442216873169


In [16]:
# Example vectors
vec1 = model.encode("Noodles are good")
vec2 = model.encode("Noodles are very bad")
print(vec1.shape,vec2.shape)


# Calculate cosine similarity
similarity = cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))
print(f"Cosine Similarity: {similarity[0][0]}")

(384,) (384,)
Cosine Similarity: 0.857244610786438


## Transfer Learning: Training a sentiment classification model

In [17]:
x

0                      Pizza is great and I love pizza.
1              I hate burger and its bad to eat burger.
2                                  I hate dirty tables.
3     Burger is amazing and I love it more than anyt...
4                   My boss is a monster and I hate him
5     The food was delivered late and I hate late de...
6            My wife love pizza and burger more than me
7           the table was bad and dirty and i hate this
8                      Food was delicious and I love it
9               It great to have good food at good time
10                               icecream was not good.
11                                icecream was not bad.
12                                   Pizza was not bad.
13                                      Burger was bad.
14                                  juice was not good.
15                                      juice was good.
16                                     ketchup was bad.
17                                    ketchup wa

In [18]:
# Convert texts to embeddings
X = model.encode(x)
y = np.array(y)

In [29]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(150, 50), max_iter=5000)
clf.fit(X, y)

MLPClassifier(hidden_layer_sizes=(150, 50), max_iter=5000)

## Evaluation on sample data

In [30]:
ip = "Noodles are good"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Positive'], dtype='<U8')

In [31]:
ip = "Noodles are not good"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype='<U8')

In [32]:
ip = "Noodles are bad"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype='<U8')

In [35]:
ip = "Noodles are not bad"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Positive'], dtype='<U8')

In [36]:
ip = "Noodles are great"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Positive'], dtype='<U8')

In [37]:
ip = "Noodles are not great"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype='<U8')

In [38]:
ip = "Noodles are not delicious"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype='<U8')

In [39]:
ip = "Noodles are delicious"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Positive'], dtype='<U8')